In [76]:
import torch
import pandas as pd

from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt
import pickle   
from collections import Counter


# Data loading and preparation

In [167]:
d = 20
edge_path = f"../data/interim/edges_xenium{d}.csv"
edges = pd.read_csv(edge_path)

/tmp/ipykernel_25707/251506899.py:3: DtypeWarning: Columns (1,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  edges = pd.read_csv(edge_path)


In [168]:
edges[edges["type"] == "sc2xen"][["node1","node2"]].shape[0] == edges[edges["type"] == "sc2xen"][["node1","node2"]].drop_duplicates().shape[0]

True

In [169]:
edges[edges["type"] == "xen2grid"][["node1","node2"]].shape[0] == edges[edges["type"] == "xen2grid"][["node1","node2"]].drop_duplicates().shape[0]

True

In [170]:
edges[["node1","node2"]].shape[0] == edges[["node1","node2"]].drop_duplicates().shape[0]

True

In [171]:
overcl = pd.read_csv("../data/interim/clones_over.csv")

In [172]:
overcl

,cell.names,leiden
0,CGCAAGGGTTAGTGAG-1,16
1,CAAGACTAGCTTGAGG-1,16
2,GCCAATACATTGCCTG-1,10
3,CTTCAATAGATGATTG-1,10
4,GATGCAAGTATTGCGT-1,10
...,...,...
10716,TCTAACCCAGGTATTG-1,2
10717,TCAACTAGTTAGCGCC-1,diploid
10718,CAGGTCAAGACCATTT-1,2
10719,TGGTGCGCACCATCCA-1,2


In [173]:
overcl.columns = ["node1","clone"]

In [174]:
overcl.clone.unique()

array(['16', '10', '0', 'diploid', '12', '8', '13', '14', '1', '7', '15',
       '4', '6', '9', '5', '11', '2', '3'], dtype=object)

In [175]:
cell_type = pd.read_excel("../data/raw/Requested_Cell_Barcode_Type_Matrices.xlsx", sheet_name="scFFPE-Seq")

In [176]:
cell_type.columns = ["node1","cell_type"]

In [177]:
Counter(cell_type.cell_type)

Counter({'Stromal': 2611,
         'Macrophages 1': 2964,
         'Perivascular-Like': 285,
         'Myoepi ACTA2+': 1235,
         'CD4+ T Cells': 2899,
         'DCIS 2': 1863,
         'Prolif Invasive Tumor': 1329,
         'Invasive Tumor': 4568,
         'CD8+ T Cells': 1843,
         'Endothelial': 1055,
         'Macrophages 2': 760,
         'DCIS 1': 2159,
         'B Cells': 1463,
         'Stromal & T Cell Hybrid': 426,
         'T Cell & Tumor Hybrid': 1003,
         'IRF7+ DCs': 210,
         'Mast Cells': 92,
         'LAMP3+ DCs': 103,
         'Myoepi KRT15+': 604})

In [178]:
overcl = cell_type.merge(overcl, on = "node1", how = "left")

In [179]:
overcl

,node1,cell_type,clone
0,AAACAAGCAAACGGGA-1,Stromal,NaN
1,AAACAAGCAAATAGGA-1,Macrophages 1,11
2,AAACAAGCAACAAGTT-1,Perivascular-Like,NaN
3,AAACAAGCAACCATTC-1,Myoepi ACTA2+,2
4,AAACAAGCAACTAAAC-1,Myoepi ACTA2+,10
...,...,...,...
27467,TTTGTGAGTGGTTACT-1,DCIS 1,11
27468,TTTGTGAGTGTTCCAG-1,Stromal,NaN
27469,TTTGTGAGTTACTTCT-1,Prolif Invasive Tumor,1
27470,TTTGTGAGTTGTCATA-1,DCIS 2,NaN


In [180]:
edges = edges.drop(columns = ["clone","celltype_major","celltype_minor"])


In [181]:
overcl = edges[edges["type"] == "sc2xen"].merge(overcl,left_on = "node1",right_on ="node1",how = "left")

In [182]:
overcl.clone = overcl.clone.fillna("diploid")

In [183]:
overcl = overcl[["node1","clone", "cell_type"]].drop_duplicates()

In [184]:
overcl.cell_type.fillna("unknown", inplace = True)

In [185]:
overcl

,node1,clone,cell_type
0,ACAAACCAGTAGCCTC-1,diploid,DCIS 2
1,ATTTAGCTCTGGAACG-1,5,DCIS 2
2,CCATAACGTATTGACC-1,diploid,unknown
3,CCCACCTTCTGGTCTA-1,diploid,DCIS 1
4,GCATGGATCGATTACT-1,diploid,DCIS 2
...,...,...,...
1582945,GACAACTAGGATTAGC-1,diploid,unknown
1624967,TTGAATCTCGCCTAGT-1,diploid,Endothelial
1634857,AAGTTCGCATTGTACC-1,diploid,Macrophages 2
1634864,GTTCGTTCACTAAGCA-1,diploid,Macrophages 2


In [186]:
Counter(overcl.cell_type)

Counter({'DCIS 2': 1839,
         'unknown': 2274,
         'DCIS 1': 2122,
         'T Cell & Tumor Hybrid': 984,
         'Invasive Tumor': 4309,
         'Myoepi KRT15+': 594,
         'CD8+ T Cells': 1808,
         'Macrophages 1': 2936,
         'Myoepi ACTA2+': 1212,
         'Stromal': 2548,
         'CD4+ T Cells': 2797,
         'Endothelial': 1028,
         'IRF7+ DCs': 210,
         'B Cells': 1387,
         'Perivascular-Like': 272,
         'Macrophages 2': 732,
         'Prolif Invasive Tumor': 1286,
         'Stromal & T Cell Hybrid': 424,
         'Mast Cells': 60,
         'LAMP3+ DCs': 102})

In [187]:
edges.merge(overcl[["clone","node1", "cell_type"]],left_on = "node1",right_on ="node1",how = "left", validate = 'm:1')

,Unnamed: 0,node1,node2,weight,type,distance,clone,cell_type
0,0,ACAAACCAGTAGCCTC-1,1,0.941976,sc2xen,NaN,diploid,DCIS 2
1,1,ATTTAGCTCTGGAACG-1,1,0.981881,sc2xen,NaN,5,DCIS 2
2,2,CCATAACGTATTGACC-1,1,0.954240,sc2xen,NaN,diploid,unknown
3,3,CCCACCTTCTGGTCTA-1,1,0.973575,sc2xen,NaN,diploid,DCIS 1
4,4,GCATGGATCGATTACT-1,1,0.956508,sc2xen,NaN,diploid,DCIS 2
...,...,...,...,...,...,...,...,...
4992483,5009635,167780,16,0.996593,xen2grid,20.0,NaN,NaN
4992484,5009636,167780,17,0.996535,xen2grid,20.0,NaN,NaN
4992485,5009637,167780,18,0.996529,xen2grid,20.0,NaN,NaN
4992486,5009638,167780,19,0.996515,xen2grid,20.0,NaN,NaN


In [188]:
edges= edges.merge(overcl[["clone","node1", "cell_type"]],left_on = "node1",right_on ="node1",how = "left")

In [189]:
edges[edges["type"] == "sc2xen"][["node1","node2"]].shape[0] == edges[edges["type"] == "sc2xen"][["node1","node2"]].drop_duplicates().shape[0]

True

In [190]:
edges[edges["type"] =="xen2grid"][["node1","node2"]].shape[0] == edges[edges["type"] == "xen2grid"][["node1","node2"]].drop_duplicates().shape[0]

True

In [191]:
edges[["node1","node2"]].shape[0] == edges[["node1","node2"]].drop_duplicates().shape[0]

True

In [192]:
emb_xen = pd.read_csv("../data/interim/embedding_xenium_scvi.csv", index_col = 0)
emb_rna = pd.read_csv("../data/interim/embedding_rna2xen_scvi.csv", index_col = 0)

In [193]:
all_nodes_graph = set(edges.node1.to_list() + edges.node2.to_list())
all_nodes_emb = set(emb_xen.index).union(set(emb_rna.index))
all_nodes = list(all_nodes_graph.intersection(all_nodes_emb))
node_encoder = {all_nodes[i]:i for i in range(len(all_nodes))}
emb_xen = emb_xen.loc[emb_xen.index.isin(all_nodes)]
emb_rna = emb_rna.loc[emb_rna.index.isin(all_nodes)]

In [194]:
edges = edges[edges.node1.isin(all_nodes)]

In [195]:
edges = edges[edges.node2.isin(all_nodes)]

In [196]:
edges.node1 = edges.node1.map(node_encoder)

In [197]:
edges.node2 = edges.node2.map(node_encoder)

In [198]:
edges

,Unnamed: 0,node1,node2,weight,type,distance,clone,cell_type
0,0,171444,0,0.941976,sc2xen,NaN,diploid,DCIS 2
1,1,173744,0,0.981881,sc2xen,NaN,5,DCIS 2
2,2,193050,0,0.954240,sc2xen,NaN,diploid,unknown
3,3,187820,0,0.973575,sc2xen,NaN,diploid,DCIS 1
4,4,192739,0,0.956508,sc2xen,NaN,diploid,DCIS 2
...,...,...,...,...,...,...,...,...
4992483,5009635,165733,15,0.996593,xen2grid,20.0,NaN,NaN
4992484,5009636,165733,16,0.996535,xen2grid,20.0,NaN,NaN
4992485,5009637,165733,17,0.996529,xen2grid,20.0,NaN,NaN
4992486,5009638,165733,18,0.996515,xen2grid,20.0,NaN,NaN


In [199]:
emb_vis = emb_xen.rename(index = node_encoder)
emb_rna = emb_rna.rename(index = node_encoder)

In [200]:
edge_index = torch.tensor([edges.node1,
                           edges.node2], dtype=torch.long)

In [201]:
edge_index

tensor([[171444, 173744, 193050,  ..., 165733, 165733, 165733],
        [     0,      0,      0,  ...,     17,     18,     19]])

In [202]:
edge_weight = torch.tensor(edges.weight.values, dtype=torch.float)

In [203]:
features = pd.concat([emb_vis, emb_rna])

In [204]:
features = features.sort_index()

In [205]:
x = torch.tensor(features.values , dtype=torch.float)

In [206]:
edges[edges.type == "xen2grid"]

,Unnamed: 0,node1,node2,weight,type,distance,clone,cell_type
1638400,1655552,75,12,0.997022,xen2grid,20.0,NaN,NaN
1638401,1655553,75,13,0.996847,xen2grid,20.0,NaN,NaN
1638402,1655554,75,14,0.996823,xen2grid,20.0,NaN,NaN
1638403,1655555,75,15,0.996770,xen2grid,20.0,NaN,NaN
1638404,1655556,75,16,0.996713,xen2grid,20.0,NaN,NaN
...,...,...,...,...,...,...,...,...
4992483,5009635,165733,15,0.996593,xen2grid,20.0,NaN,NaN
4992484,5009636,165733,16,0.996535,xen2grid,20.0,NaN,NaN
4992485,5009637,165733,17,0.996529,xen2grid,20.0,NaN,NaN
4992486,5009638,165733,18,0.996515,xen2grid,20.0,NaN,NaN


In [207]:
edges.clone = edges.clone.fillna("missing")

In [208]:
edges[edges.clone == "missing"]

,Unnamed: 0,node1,node2,weight,type,distance,clone,cell_type
1638400,1655552,75,12,0.997022,xen2grid,20.0,missing,NaN
1638401,1655553,75,13,0.996847,xen2grid,20.0,missing,NaN
1638402,1655554,75,14,0.996823,xen2grid,20.0,missing,NaN
1638403,1655555,75,15,0.996770,xen2grid,20.0,missing,NaN
1638404,1655556,75,16,0.996713,xen2grid,20.0,missing,NaN
...,...,...,...,...,...,...,...,...
4992483,5009635,165733,15,0.996593,xen2grid,20.0,missing,NaN
4992484,5009636,165733,16,0.996535,xen2grid,20.0,missing,NaN
4992485,5009637,165733,17,0.996529,xen2grid,20.0,missing,NaN
4992486,5009638,165733,18,0.996515,xen2grid,20.0,missing,NaN


In [209]:
nodes_atr = edges[["node1","type","cell_type", "clone"]].drop_duplicates()

In [210]:
nodes_atr = nodes_atr.sort_values(by = "node1")

In [211]:
nodes_atr.clone.unique()

array(['missing', 'diploid', '1', '9', '4', '7', '8', '12', '2', '0', '6',
       '5', '11', '16', '15', '10', '14', '13', '3'], dtype=object)

In [212]:
nodes_atr.cell_type = nodes_atr.cell_type.fillna("missing")

In [213]:
nodes_atr

,node1,type,cell_type,clone
1638400,75,xen2grid,missing,missing
1638408,76,xen2grid,missing,missing
1638428,77,xen2grid,missing,missing
1638448,78,xen2grid,missing,missing
1638468,79,xen2grid,missing,missing
...,...,...,...,...
33502,194323,sc2xen,CD4+ T Cells,diploid
7324,194324,sc2xen,Myoepi KRT15+,diploid
128187,194325,sc2xen,CD8+ T Cells,diploid
10958,194326,sc2xen,DCIS 1,7


In [214]:
from collections import Counter

Counter(nodes_atr.cell_type)

Counter({'missing': 165329,
         'CD4+ T Cells': 2797,
         'Prolif Invasive Tumor': 1286,
         'Myoepi KRT15+': 594,
         'Stromal': 2548,
         'DCIS 1': 2122,
         'Invasive Tumor': 4309,
         'unknown': 2274,
         'CD8+ T Cells': 1808,
         'Macrophages 1': 2936,
         'Endothelial': 1028,
         'B Cells': 1387,
         'Stromal & T Cell Hybrid': 424,
         'DCIS 2': 1839,
         'IRF7+ DCs': 210,
         'Macrophages 2': 732,
         'T Cell & Tumor Hybrid': 984,
         'Myoepi ACTA2+': 1212,
         'Perivascular-Like': 272,
         'LAMP3+ DCs': 102,
         'Mast Cells': 60})

In [215]:
ct_list = list(nodes_atr.cell_type.unique())
ct_list.remove("missing")

In [216]:
Counter(nodes_atr.clone)

Counter({'missing': 165329,
         'diploid': 19533,
         '1': 1234,
         '9': 479,
         '4': 786,
         '7': 597,
         '8': 583,
         '12': 315,
         '2': 896,
         '0': 1502,
         '6': 641,
         '5': 809,
         '11': 361,
         '16': 61,
         '15': 67,
         '10': 510,
         '14': 224,
         '13': 304,
         '3': 22})

In [217]:
clone_list = list(nodes_atr.clone.unique())
clone_list.remove("missing")
clone_list.remove("diploid")
clone_dict = {x:int(x) for x in clone_list}
clone_dict["missing"] = -1
clone_dict["diploid"] = len(clone_dict)-1

In [218]:
clone_dict

{'1': 1,
 '9': 9,
 '4': 4,
 '7': 7,
 '8': 8,
 '12': 12,
 '2': 2,
 '0': 0,
 '6': 6,
 '5': 5,
 '11': 11,
 '16': 16,
 '15': 15,
 '10': 10,
 '14': 14,
 '13': 13,
 '3': 3,
 'missing': -1,
 'diploid': 17}

In [219]:
    ct_list = list(nodes_atr.cell_type.unique())
    ct_list.remove("missing")

In [220]:
type_dict = {ct_list[i]:i for i in range(len(ct_list))}

In [221]:
type_dict["missing"] = -1

In [222]:
type_dict

{'CD4+ T Cells': 0,
 'Prolif Invasive Tumor': 1,
 'Myoepi KRT15+': 2,
 'Stromal': 3,
 'DCIS 1': 4,
 'Invasive Tumor': 5,
 'unknown': 6,
 'CD8+ T Cells': 7,
 'Macrophages 1': 8,
 'Endothelial': 9,
 'B Cells': 10,
 'Stromal & T Cell Hybrid': 11,
 'DCIS 2': 12,
 'IRF7+ DCs': 13,
 'Macrophages 2': 14,
 'T Cell & Tumor Hybrid': 15,
 'Myoepi ACTA2+': 16,
 'Perivascular-Like': 17,
 'LAMP3+ DCs': 18,
 'Mast Cells': 19,
 'missing': -1}

In [223]:
nodes_atr.clone = nodes_atr.clone.map(clone_dict)

In [224]:
edges.cell_type = edges.cell_type.fillna("missing")

In [225]:
nodes_atr.cell_type = nodes_atr.cell_type.map(type_dict)

In [226]:
nodes_atr = nodes_atr.set_index("node1")

In [227]:
nodes_atr.head()

,type,cell_type,clone
node1,,,
75,xen2grid,-1,-1
76,xen2grid,-1,-1
77,xen2grid,-1,-1
78,xen2grid,-1,-1
79,xen2grid,-1,-1


In [228]:
nodes_atr["type"].unique()

array(['xen2grid', 'sc2xen'], dtype=object)

In [229]:
nodes_atr[nodes_atr["type"] == "sc2xen"]

,type,cell_type,clone
node1,,,
613,sc2xen,0,17
635,sc2xen,1,1
3236,sc2xen,2,17
3348,sc2xen,0,17
3356,sc2xen,3,17
...,...,...,...
194323,sc2xen,0,17
194324,sc2xen,2,17
194325,sc2xen,7,17


In [230]:
features = features.join(nodes_atr)

In [231]:
features.clone = features.clone.fillna(-1)

In [232]:
features.cell_type = features.cell_type.fillna(-1)

In [233]:
features

,0,1,type,cell_type,clone
0,0.747124,0.243280,NaN,-1.0,-1.0
1,0.821959,0.419919,NaN,-1.0,-1.0
2,0.453010,0.237194,NaN,-1.0,-1.0
3,0.977938,0.125345,NaN,-1.0,-1.0
4,0.542644,0.224116,NaN,-1.0,-1.0
...,...,...,...,...,...
194323,-0.862637,1.086466,sc2xen,0.0,17.0
194324,0.568115,1.324937,sc2xen,2.0,17.0
194325,-0.895562,1.305892,sc2xen,7.0,17.0
194326,1.481940,0.205432,sc2xen,4.0,7.0


In [234]:
y_clone = torch.tensor(features.clone.values,dtype=torch.long)


In [235]:
y_type = torch.tensor(features.cell_type.values,dtype=torch.long)

In [236]:
y_type.unique()

tensor([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19])

In [237]:
y_clone.unique()

tensor([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17])

In [238]:
data = Data(x=x, edge_index=edge_index, y_clone = y_clone,y_type = y_type, 
edge_type = edges.type.values, edge_attr = edge_weight)

In [239]:
data.validate(raise_on_error=True)


True

In [240]:
full_dict = {}
full_dict["nodes"] = node_encoder
full_dict["clones"] = clone_dict
full_dict["types"] = type_dict


In [241]:

    torch.save(data, f"../data/processed/data_xen{d}.pt")
    with open(f'../data/processed/full_encoding_xen{d}.pkl', 'wb') as fp:
        pickle.dump(full_dict, fp)